After working with the USGS Data some questions were raised about null values from 1978 to 2003, which resulted in getting an additional data source from the PR Seismic Network page: http://www.prsn.uprm.edu/Spanish/catalogue/index.php.

Data is saved into a csv file then loaded and cleaned in python and finally saved in the same MySQL database.

In [2]:
from etl import connect_to_db, get_intersections, clean_data, load_to_db, scrap_prsn, load_delta_data

In [3]:
db = 'eq_pr'
user = 'root'
conn = connect_to_db(db,user)

Enter database password:········
Successfull connection to MySQL database


# Create table to save data into MySQL

In [ ]:
#Create table for Earthquake Data 
conn.execute('''CREATE TABLE `eq_pr`.`prsn_data` (
  `id` VARCHAR(45) NOT NULL,
  `date_time` DATETIME NOT NULL,
  `lat` FLOAT NOT NULL,
  `lon` FLOAT NULL,
  `mag` FLOAT ,
  `magType` VARCHAR(10) ,
  `sources` VARCHAR(45) ,
  `depth` FLOAT ,
  `region` VARCHAR(45) NULL,
  PRIMARY KEY (`id`),
  INDEX `id` (`id` ASC) VISIBLE);
''').rowcount

# ETL Process 

In [ ]:
##Initial Load by year

years = list(range(1986,2020))

for y in years:

    #set parameters for start datetime 
    syear = str(y)
    smonth = '1'
    sday = '1'
    shour = '0'
    smins = '0'
    #set parameters for end datetime 
    eyear = str(y)
    emonth = '12'
    eday = '31'
    ehour = '23'
    emins = '59'

    df=scrap_prsn_initial(syear,smonth,sday,shour,smins,eyear,emonth,eday,ehour,emins)
    prsn_gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon,df.lat))
    df = get_intersections(prsn_gdf,conn)
    #Saving data into MySQL
    load_to_db(df, conn, 'PRSN')

In [ ]:
#initial load of year 2020 since there is too much activity, doing it by day or month by month

months = list(range(9,10))
m_30 = [4,6,9,11]
m_31 = [1,2,3,5,7,8,10,12]


for m in months:
    if m in (m_30):
        ed = 30
    elif m == 2:
        ed = 29
    elif m in (m_31):
        ed = 31
    #set parameters for start datetime 
    syear = '2020'
    smonth = str(m)
    sday = '1'
    shour = '0'
    smins = '0'
    #set parameters for end datetime 
    eyear = '2020'
    emonth = str(m)
    eday = str(ed)
    ehour = '23'
    emins = '59'

    df=scrap_prsn(syear,smonth,sday,shour,smins,eyear,emonth,eday,ehour,emins)
    prsn_gdf = clean_data(df, 'PRSN')
    df = get_intersections(prsn_gdf,conn)
    #Saving data into MySQL
    load_to_db(df, conn, 'PRSN')

In [4]:
#delta load to run going foward

load_delta_data(conn, 'PRSN')

Start date =  2020-10-15 22:04:14
End date =  2020-10-15 20:04:01.505280
Number of records: 1
Assigning regions...
Validating region assignment...
No issues with region assignmet!
Loading data into MySQL...
No records to update
